In [43]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [12]:
train_raw = pd.read_csv('train_MPRA.txt', delimiter='\t', header=None)
test_raw = pd.read_csv('test_MPRA.txt', delimiter='\t', header=None)
train_sol = pd.read_csv('trainsolutions.txt', delimiter='\t', header=None)
train_raw.head()
strand_length = 295


In [13]:
# Get our x and y data
train_scores = np.array(train_raw.iloc[:, 2:297]) #Dimensions are 8000 (samples) by 295 (SHARPR scores per nucleotide)
raw_dna_strands_train = [list(train_raw[1][i]) for i in range(len(train_raw))] #List of lists holding DNA strands separated by character. Size 8000 lists each of length 290
embedded_dna_strands_train = [np.column_stack((np.array(pd.get_dummies(pd.concat([pd.Series(raw_dna_strands_train[i]), pd.Series(["A", "C", "T", "G"])]), dtype='int'))[:-4], np.arange(295))) for i in range(len(train_raw))] #One hot encoded dna strands, list of 8000 matrices, each (295,5)
embedded_dna_strands_train = [embedded_dna_strands_train[i] for i in range(len(embedded_dna_strands_train)) if not ("N" in raw_dna_strands_train[i])]
train_scores  = [train_scores[i] for i in range(len(raw_dna_strands_train)) if not ("N" in raw_dna_strands_train[i])]
#Repeat for test data
raw_dna_strands_test = [list(test_raw[1][i]) for i in range(len(test_raw))] #List of lists holding DNA strands separated by character. Size 8000 lists each of length 290
embedded_dna_strands_test = [np.column_stack((np.array(pd.get_dummies(pd.concat([pd.Series(raw_dna_strands_test[i]), pd.Series(["A", "C", "T", "G"])]), dtype='int'))[:-4], np.arange(295))) for i in range(len(test_raw))]
embedded_dna_strands_test = [embedded_dna_strands_test[i] for i in range(len(embedded_dna_strands_test)) if not ("N" in raw_dna_strands_test[i])]

In [14]:
#Add column with unique identifier for each nucleotide (sequence + location)
train_sol[3] = [str(train_sol.iloc[i, 1][5:]).zfill(4) + str(train_sol.iloc[i,2]).zfill(3) for i in range(len(train_sol))]

#Split by activators and repressors
train_sol_act = train_sol[train_sol[0] == 'A'][3]
train_sol_rep = train_sol[train_sol[0] == 'R'][3]

In [15]:

"N" in raw_dna_strands_train[3356]

True

In [16]:
raw_dna_strands_train[3356]

['N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'G',
 'A',
 'A',
 'T',
 'T',
 'C',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'G',
 'A',
 'C',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'C',
 'T',
 'G',
 'T',
 'G',
 'G',
 'G',
 'G',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'G',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'T',
 'A',
 'G',
 'T',
 'T',
 'G',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'G',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'A',
 'T',
 'G',
 'C',
 'G',
 'A',
 'T',
 'G',
 'G',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'T',
 'G',
 'A',
 'G',
 'C',
 'A',
 'G',
 'A',
 'G',
 'T',
 'G',
 'A',
 'A',
 'G',
 'T',
 'G',
 'G',
 'A',
 'A',
 'A',
 'G',
 'G',
 'T',
 'G',
 'T',
 'A',
 'G',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'A',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T',
 'G',
 'G',
 'A',
 'G',
 'T'

### ML Model

In [44]:
class DNADataset(Dataset):
    def __init__(self, embedded_dna_strands, train_scores):
        self.x = torch.tensor(embedded_dna_strands, dtype=torch.float32) # Convert x and y to tensors
        self.y = torch.tensor(train_scores, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

class SelfAttentionFeedForward(nn.Module):
    #Initialize hyperparameters and NN matrices
    def __init__(self, attention_size, embed_size, hidden_size, hidden_layers, lr):
        super().__init__()
        self.attention_size = attention_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.hidden_layers = hidden_layers
        self.lr = lr

        self.initAttention()
        self.initFFN()

    #Initialize our weight matrices as torch objects, allows them to be automatically optimized
    def initAttention(self):
        self.W_Q = nn.Parameter(torch.rand(self.attention_size, self.embed_size))
        self.W_K = nn.Parameter(torch.rand(self.attention_size, self.embed_size))
        self.W_V = nn.Parameter(torch.rand(self.attention_size, self.embed_size))
        self.b = nn.Parameter(torch.rand(295)) # This is an addition term, analogous to y-intercept

        self.optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.lr,
            amsgrad=True,
        )
    
    #Initialize Feed Forward layers, based on however many hidden layers we want
    def initFFN(self):
        fc1 = nn.Linear(self.attention_size, self.hidden_size)
        relu1 = nn.ReLU()
        self.layers = [fc1, relu1]
        for i in range(self.hidden_layers - 1):
            self.layers.append(nn.Linear(self.hidden_size, self.hidden_size))
            self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(self.hidden_size, 1))


    def loss(self, predicted, y):
        return torch.norm(predicted - y)

    def predict(self, x):
        # Multiply by all embeddings
        queries = torch.matmul(self.W_Q, x.T)
        keys = torch.matmul(self.W_K, x.T)
        values = torch.matmul(self.W_V, x.T)

        #Compute attention and then normalize
        attention = torch.matmul(queries, keys.T)
        weights = torch.nn.functional.softmax(attention, dim=0)

        # Use as weights for values
        contextualized_embeddings = torch.matmul(weights, values).T #This is dim self.attention_size x 295
        # Run through all FFN layers
        for layer in self.layers:
            contextualized_embeddings = layer(contextualized_embeddings)
        # Return prediction with added b term
        return contextualized_embeddings + self.b

    def train_step(self, x, y):
        pred = self.predict(x)
        loss = self.loss(pred, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return None

    def train(self, dataloader):
        for x_batch, y_batch in dataloader:
            self.train_step(x_batch, y_batch)



In [47]:
# Create the dataset
dataset = DNADataset(embedded_dna_strands_train, train_scores)

# Create a DataLoader for batching, shuffling, and parallel data loading
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = SelfAttentionFeedForward(50, 5, 20, 3, 1e-5)
model.train(dataloader)


/var/folders/98/h70r_yzx4qxdpyxrq5f91p7c0000gn/T/ipykernel_55812/1527061728.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_1aidzjezue/croot/pytorch_1687856425340/work/torch/csrc/utils/tensor_new.cpp:248.)
  self.x = torch.tensor(embedded_dna_strands, dtype=torch.float32)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (160x295 and 5x50)

In [42]:
model.predict(torch.Tensor(embedded_dna_strands_train[0])).detach().numpy() - train_scores[0]

array([[ 0.22174352,  0.71485429,  0.62327915, ...,  1.0669322 ,
         0.41677427,  1.01910806],
       [ 0.27477124,  0.76788204,  0.6763069 , ...,  1.11995983,
         0.46980199,  1.07213569],
       [ 0.34722375,  0.84033453,  0.74875938, ...,  1.19241238,
         0.54225451,  1.14458823],
       ...,
       [19.20031839, 19.69342905, 19.60185342, ..., 20.04550743,
        19.3953495 , 19.99768257],
       [19.26924997, 19.76236063, 19.670785  , ..., 20.11443901,
        19.46428108, 20.06661415],
       [19.33420473, 19.82731538, 19.73573976, ..., 20.17939377,
        19.52923584, 20.13156891]])

In [20]:
train_scores[0]

array([ 0.019,  0.019,  0.019,  0.018,  0.017,  0.017,  0.016,  0.015,
        0.037,  0.059,  0.082,  0.104,  0.126,  0.125,  0.125,  0.124,
        0.124,  0.123,  0.116,  0.109,  0.102,  0.095,  0.088,  0.07 ,
        0.053,  0.035,  0.018,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.005,  0.01 ,  0.014,  0.019,  0.024,  0.019,  0.014,
        0.01 ,  0.005,  0.   , -0.052, -0.104, -0.156, -0.208, -0.26 ,
       -0.27 , -0.28 , -0.29 , -0.3  , -0.31 , -0.298, -0.286, -0.275,
       -0.263, -0.251, -0.253, -0.255, -0.258, -0.26 , -0.262, -0.248,
       -0.234, -0.219, -0.205, -0.191, -0.202, -0.213, -0.223, -0.234,
       -0.245, -0.235, -0.225, -0.214, -0.204, -0.194, -0.191, -0.188,
       -0.185, -0.182, -0.179, -0.181, -0.183, -0.184, -0.186, -0.188,
       -0.155, -0.122, -0.089, -0.056, -0.023, -0.007,  0.009,  0.025,
        0.041,  0.057,  0.086,  0.116,  0.145,  0.175,  0.204,  0.21 ,
        0.216,  0.222,  0.228,  0.234,  0.24 ,  0.246,  0.252,  0.258,
      